In [1]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 드라이버 설정 및 사이트 접속
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://redbutton.co.kr/")
time.sleep(0.5)

# 팝업 닫고 "게임 검색하기" 버튼 클릭
try:
    not_today = driver.find_element(By.CLASS_NAME, "jt_popup_notoday")
    ActionChains(driver).click(not_today).perform()
except:
    pass  # 팝업 없을 수도 있음

buttons = driver.find_elements(By.CLASS_NAME, "elementor-button-text")
ActionChains(driver).click(buttons[1]).perform()
time.sleep(0.5)

# 게임 상세정보 저장 리스트
game_data_list = []

# 지역 드롭다운 요소 선택
dropdown_location = driver.find_element(By.ID, "game-location")
select_location = Select(dropdown_location)
locations_values = [opt.get_attribute("value") for opt in select_location.options if opt.get_attribute("value")]

# 각 지역마다 순회
for location_value in locations_values:
    try:
        dropdown_location = driver.find_element(By.ID, "game-location")
        select_location = Select(dropdown_location)
        select_location.select_by_value(location_value)
        time.sleep(1)

        selected_location = select_location.first_selected_option
        location_name = selected_location.text.strip()
        print(f"\n===== 🔍 {location_name} 지역 처리 중... =====")

        dropdown_store = driver.find_element(By.ID, "game-store")
        select_store = Select(dropdown_store)

        for option in select_store.options:
            if not option.get_attribute("value"):
                continue

            try:
                dropdown_store = driver.find_element(By.ID, "game-store")
                select_store = Select(dropdown_store)
                select_store.select_by_value(str(option.get_attribute("value")))
                time.sleep(1)

                selected_store = select_store.first_selected_option
                store_name = selected_store.text.strip()
                print(f"🔍 {store_name} 지점 처리 중...")

                keyword_input = driver.find_element(By.ID, "game-keyword")
                ActionChains(driver).send_keys_to_element(keyword_input, Keys.ENTER).perform()
                time.sleep(1)

                game_infos = driver.find_elements(By.CLASS_NAME, "game-info")

                for game in game_infos:
                    try:
                        title = game.find_element(By.CLASS_NAME, "game-title").text.strip()
                    except:
                        continue

                    difficulty, players, playtime = "", "", ""
                    has_video = "아니오"
                    keywords = []

                    content_rules = game.find_elements(By.CLASS_NAME, "content-rule")
                    for rule in content_rules:
                        try:
                            label = rule.find_element(By.TAG_NAME, "b").text.strip()
                            value = rule.text.replace(label, "").strip()
                            if label == "난이도":
                                difficulty = value
                            elif label == "인원":
                                players = value
                            elif label == "게임시간":
                                playtime = value
                        except:
                            continue

                    try:
                        guide_span = game.find_element(By.CLASS_NAME, "content-guide")
                        if "설명영상 보유" in guide_span.text:
                            has_video = "예"
                    except:
                        pass

                    keywords = [kw.text.strip() for kw in game.find_elements(By.CLASS_NAME, "content-keyword")]

                    game_data_list.append({
                        "지역": location_name,
                        "지점": store_name,
                        "게임명": title,
                        "난이도": difficulty,
                        "인원": players,
                        "게임시간": playtime,
                        "설명영상": has_video,
                        "키워드": ", ".join(keywords)
                    })
                print(f"✅ {store_name} - {len(game_infos)}개 게임 수집 완료\n")

            except Exception as e:
                print(f"❌ 지점 처리 중 오류 발생: {e}")

    except Exception as e:
        print(f"❌ 지역 value={location_value} 처리 중 오류 발생: {e}")

# 드라이버 종료
driver.quit()

# 요약 출력
print(f"\n🎯 전체 게임 수집 완료: {len(df)}개 게임")


===== 🔍 서울 지역 처리 중... =====
🔍 신촌 지점 처리 중...
✅ 신촌 - 414개 게임 수집 완료
🔍 신림 지점 처리 중...
✅ 신림 - 384개 게임 수집 완료
🔍 대학로 지점 처리 중...
✅ 대학로 - 425개 게임 수집 완료
🔍 강남 지점 처리 중...
✅ 강남 - 407개 게임 수집 완료
🔍 노원 지점 처리 중...
✅ 노원 - 346개 게임 수집 완료
🔍 홍대 지점 처리 중...
✅ 홍대 - 294개 게임 수집 완료
🔍 잠실새내 지점 처리 중...
✅ 잠실새내 - 314개 게임 수집 완료
🔍 연신내 지점 처리 중...
✅ 연신내 - 347개 게임 수집 완료
🔍 강남2호 지점 처리 중...
✅ 강남2호 - 341개 게임 수집 완료
🔍 수유역 지점 처리 중...
✅ 수유역 - 310개 게임 수집 완료
🔍 이수역 지점 처리 중...
✅ 이수역 - 346개 게임 수집 완료
🔍 마곡발산역 지점 처리 중...
✅ 마곡발산역 - 317개 게임 수집 완료
🔍 한양대 지점 처리 중...
✅ 한양대 - 384개 게임 수집 완료
🔍 서울대입구역 지점 처리 중...
✅ 서울대입구역 - 323개 게임 수집 완료
🔍 성신여대 지점 처리 중...
✅ 성신여대 - 322개 게임 수집 완료
🔍 성수 지점 처리 중...
✅ 성수 - 300개 게임 수집 완료
🔍 종로 지점 처리 중...
✅ 종로 - 373개 게임 수집 완료
🔍 인시네마강남 지점 처리 중...
✅ 인시네마강남 - 307개 게임 수집 완료
🔍 방이 지점 처리 중...
✅ 방이 - 293개 게임 수집 완료
🔍 영등포 지점 처리 중...
✅ 영등포 - 314개 게임 수집 완료
🔍 강남논현 지점 처리 중...
✅ 강남논현 - 517개 게임 수집 완료
🔍 압구정로데오 지점 처리 중...
✅ 압구정로데오 - 278개 게임 수집 완료
🔍 망원역 지점 처리 중...
✅ 망원역 - 349개 게임 수집 완료
🔍 오목교점 지점 처리 중...
✅ 오목교점 - 277개 게임 수집 완료
🔍 천호로데오점 지점 처리 중...

In [ ]:
# DataFrame 변환 및 저장
df = pd.DataFrame(game_data_list)
df.to_csv("redbutton_game_info_all.csv", index=False)